In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from PIL import Image
from tqdm import tqdm

# Check available device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths to dataset and annotations
train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'
test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'


In [3]:
class AgeDataset(Dataset):
    def __init__(self, data_path, annot_path, train=True, transform=None):
        self.data_path = data_path
        self.annot_path = annot_path
        self.train = train
        self.df = pd.read_csv(annot_path)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = os.path.join(self.data_path, self.df.iloc[index]['file_id'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.train:
            age = self.df.iloc[index]['age']
            return image, age
        return image

# Get the training data age distribution
train_df = pd.read_csv(train_ann)
age_counts = train_df['age'].value_counts().to_dict()

# Calculate weights inversely proportional to age distribution
train_df['weight'] = train_df['age'].apply(lambda x: 1.0 / age_counts[x])

# Create a WeightedRandomSampler
sampler = WeightedRandomSampler(weights=train_df['weight'].tolist(), num_samples=len(train_df['weight']), replacement=True)

# The rest of your code here for defining the transformations and datasets remains the same.


# Define the data augmentations and transformations
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2)
    ], p=0.5),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the datasets
train_dataset = AgeDataset(train_path, train_ann, train=True, transform=train_transform)
test_dataset = AgeDataset(test_path, test_ann, train=False, transform=train_transform)

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


In [4]:
# Initialize a more complex model, such as ResNeXt, with fine-tuning
model = models.resnext50_32x4d(pretrained=True)

# Fine-tune additional layers
for param in model.parameters():
    param.requires_grad = False
for layer in list(model.children())[5:]:
    for param in layer.parameters():
        param.requires_grad = True

# Replace the final layer with a linear layer for age prediction
model.fc = nn.Linear(model.fc.in_features, 1)
model.to(device)

# Loss function, optimizer, and learning rate scheduler
criterion = nn.L1Loss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop with the introduction of learning rate scheduler
num_epochs = 15  # Increase if needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_mae = 0
    for images, ages in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
        images = images.to(device)
        ages = ages.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, ages)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_mae += torch.abs(outputs - ages).mean().item()

    scheduler.step()  # Update the learning rate
    epoch_loss = total_loss / len(train_loader)
    epoch_mae = total_mae / len(train_loader)
    print(f'Epoch {epoch+1}: Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f}')


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 156MB/s] 
Epoch 1/15: 100%|██████████| 334/334 [02:03<00:00,  2.70batch/s]


Epoch 1: Loss: 12.6820, MAE: 12.6820


Epoch 2/15: 100%|██████████| 334/334 [02:01<00:00,  2.74batch/s]


Epoch 2: Loss: 9.8532, MAE: 9.8532


Epoch 3/15: 100%|██████████| 334/334 [02:02<00:00,  2.73batch/s]


...

Epoch 13: Loss: 6.4924, MAE: 6.4924


Epoch 14/15: 100%|██████████| 334/334 [02:02<00:00,  2.73batch/s]


Epoch 14: Loss: 6.4526, MAE: 6.4526


Epoch 15/15: 100%|██████████| 334/334 [02:02<00:00,  2.73batch/s]

Epoch 15: Loss: 6.3053, MAE: 6.3053


In [5]:
@torch.no_grad()
def predict(loader, model):
    model.eval()
    predictions = []
    for images in tqdm(loader, desc='Predicting', unit='batch'):
        images = images.to(device)
        outputs = model(images)
        predictions.extend(outputs.view(-1).cpu().detach().numpy())
    return predictions

preds = predict(test_loader, model)

submit = pd.read_csv(test_ann)
submit['age'] = preds
submit.to_csv('submission.csv', index=False)


Predicting: 100%|██████████| 31/31 [00:11<00:00,  2.62batch/s]
